# Nissen et al., 2023: Severe 21st-century OA in Antarctic MPAs
#
# script to calculate how many grid cells lie within each MPA 
#
# note that some paths are hard-coded throughout the script

In [1]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import griddata
from matplotlib import cm
from netCDF4 import Dataset, MFDataset
import pandas as pd
from math import sin, cos, sqrt, atan2, radians
from numba import njit
from scipy.io import netcdf
from pylab import *
import math as mt

In [2]:
#---
# load mesh info
#---

path_mesh = '/pscratch/sd/c/cnissen/'
file_mesh = 'Nissen2022_FESOM_REcoM_mesh_information_corrected_20220910.nc'

f1 = Dataset(path_mesh+file_mesh) #xr.open_dataset(path+file1)
lat      = f1.variables['lat'][:]
lon      = f1.variables['lon'][:]
zlevs    = f1.variables['zlevs'][:]
cavities = f1.variables['cavity'][:]
topo     = f1.variables['topo'][:]
area     = f1.variables['cell_area'][:]
volume   = f1.variables['cell_volume'][:]
f1.close()
print(lat.shape)


(72411,)


In [3]:
#-------
# get subregions
#-------

#----
# FUNCTIONS
#----

def get_intersection(lst1, lst2):  
        # Use of hybrid method 
        temp = set(lst2) 
        lst3 = [value for value in lst1 if value in temp] 
        return lst3
    
def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

# get subareas based on lat only
#----
# FUNCTIONS
#----

def get_intersection(lst1, lst2):  
        # Use of hybrid method 
        temp = set(lst2) 
        lst3 = [value for value in lst1 if value in temp] 
        return lst3
    
def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

# get subareas based on lat only
def get_subarea_indices_sectors_lat_only(lat,lon):
    #subareas  = ['Larsen/FRIS','eWeddell','Amery/Australian','Ross Sea','Amundsen/Bellingh']
    
    lat_threshold = -60 #-60 # 0=all SH data
        
    ross1 = 150 #160
    ross2 = -130 # -145
    
    eWeddell1 = -10
    
    print ('Subareas: south of '+str(-1*lat_threshold)+'°S...')
        
    shelf_ind = np.zeros(len(lon))
    for i in range(1,len(lon)+1):
        dd = lon[i-1]
        if lat[i-1]<lat_threshold and dd>-52 and dd<eWeddell1: # Filchner-Ronne & Larsen 
            shelf_ind[i-1]=1
        elif lat[i-1]<-74 and dd>-85 and dd<eWeddell1: # Filchner-Ronne & Larsen in the very south
            shelf_ind[i-1]=1
        elif lat[i-1]<lat_threshold and dd>eWeddell1 and dd<30: # eWeddell
            shelf_ind[i-1]=6
        elif lat[i-1]<lat_threshold and dd>30 and dd<90: # Prydz
            shelf_ind[i-1]=2
        elif lat[i-1]<lat_threshold and dd>90 and dd<ross1: # Amery & Australian
            shelf_ind[i-1]=3
        elif lat[i-1]<lat_threshold and dd>ross1:  # Ross Sea 
            shelf_ind[i-1]=4
        elif lat[i-1]<lat_threshold and dd<ross2:   # Ross Sea
            shelf_ind[i-1]=4
        elif lat[i-1]<lat_threshold and dd>ross2 and dd<-69: #dd<-66: # Amundsen & Bellinghausen
            shelf_ind[i-1]=5
        elif lat[i-1]<lat_threshold and dd>-69 and dd<-52: #dd<-66: # Antarctic Peninsula
            wap_lat = -70
            if lat[i-1]>wap_lat:
                shelf_ind[i-1]=7 # pensinsula
            else:
                #shelf_ind[i-1]=7 
                # now find correct separation between Amundsen & Weddell Sea
                if dd>-66:
                    shelf_ind[i-1]=1
                else:
                    shelf_ind[i-1]=5
                    
            
        # now find correct separation between Amundsen & Weddell Sea
        #if lat[i-1]<=wap_lat:
            
                
        
 #       # now find correct separation between Amundsen & Weddell Sea
 #       if lat[i-1]<-66 and dd>-63 and dd<eWeddell1: # Filchner-Ronne & Larsen & eastern WS
 #           shelf_ind[i-1]=1
 #       if lat[i-1]<-66 and lat[i-1]>-70 and dd>-64.5 and dd<eWeddell1: # Filchner-Ronne & Larsen & eastern WS
 #           shelf_ind[i-1]=1
 #       if lat[i-1]<-66.8 and lat[i-1]>-70 and dd>-66 and dd<eWeddell1: # Filchner-Ronne & Larsen & eastern WS
 #           shelf_ind[i-1]=1
 #       if lat[i-1]<-64 and lat[i-1]>-66 and dd>-60.6 and dd<eWeddell1: # Filchner-Ronne & Larsen & eastern WS
 #           shelf_ind[i-1]=1
 #       if lat[i-1]<-64.8 and lat[i-1]>-66 and dd>-62.4 and dd<eWeddell1: # Filchner-Ronne & Larsen & eastern WS
 #           shelf_ind[i-1]=1
 #       if lat[i-1]<-64.7 and lat[i-1]>-65.9 and dd>-62.2 and dd<eWeddell1: # Filchner-Ronne & Larsen & eastern WS
 #           shelf_ind[i-1]=1
 #       if lat[i-1]<-63.4 and lat[i-1]>-64.5 and dd>-58 and dd<eWeddell1: # Filchner-Ronne & Larsen & eastern WS
 #           shelf_ind[i-1]=1
            
    return shelf_ind


#---
# plot map of subregions
#----

shelf_ind = get_subarea_indices_sectors_lat_only(lat,lon)

ind = np.where((topo>2000) & (cavities==0))[0]
shelf_ind[ind]=0
shelf_ind[cavities==1]=0

weddell = np.zeros_like(shelf_ind)
weddell[shelf_ind==1]=1

eWeddell = np.zeros_like(shelf_ind)
eWeddell[shelf_ind==6]=1

prydz = np.zeros_like(shelf_ind)
prydz[shelf_ind==2]=1

adelie = np.zeros_like(shelf_ind)
adelie[shelf_ind==3]=1

ross = np.zeros_like(shelf_ind)
ross[shelf_ind==4]=1

amundsen = np.zeros_like(shelf_ind)
amundsen[shelf_ind==5]=1

wap = np.zeros_like(shelf_ind)
wap[shelf_ind==7]=1


#----
# get masks for MPAs
#----

path1 = '/pscratch/sd/c/cnissen/HLRN_runs_postprocessed/masks/masks_MPAs/'

f1 = Dataset(path1+'MPA_mask_AntarcticPeninsula_fesom_mesh_COARZE.nc')
mask_WAP_MPA = f1.variables['mask_fesom'][:]
f1.close()
f1 = Dataset(path1+'MPA_mask_East_Antarctica_1_fesom_mesh_COARZE.nc')
mask_eastAA1_MPA = f1.variables['mask_fesom'][:]
f1.close()
f1 = Dataset(path1+'MPA_mask_East_Antarctica_2_fesom_mesh_COARZE.nc')
mask_eastAA2_MPA = f1.variables['mask_fesom'][:]
f1.close()
f1 = Dataset(path1+'MPA_mask_East_Antarctica_3_fesom_mesh_COARZE.nc')
mask_eastAA3_MPA = f1.variables['mask_fesom'][:]
f1.close()
f1 = Dataset(path1+'MPA_mask_Orkney_fesom_mesh_COARZE.nc')
mask_Orkney_MPA = f1.variables['mask_fesom'][:]
f1.close()
f1 = Dataset(path1+'MPA_mask_RossSea_fesom_mesh_COARZE.nc')
mask_RossSea_MPA = f1.variables['mask_fesom'][:]
f1.close()
f1 = Dataset(path1+'MPA_mask_WeddellSea_fesom_mesh_COARZE.nc')
mask_WeddellSea_MPA = f1.variables['mask_fesom'][:]
f1.close()

all_mpas = np.zeros_like(shelf_ind)
all_mpas[mask_WAP_MPA==1]=1
all_mpas[mask_eastAA1_MPA==1]=2
all_mpas[mask_eastAA2_MPA==1]=3
all_mpas[mask_eastAA3_MPA==1]=4
all_mpas[mask_Orkney_MPA==1]=5
all_mpas[mask_RossSea_MPA==1]=6
all_mpas[mask_WeddellSea_MPA==1]=7
all_mpas[cavities==1]=0

#------------

subareas  = ['Larsen/FRIS','eWeddell','Amery/Australian','Ross Sea','Amundsen/Bellingh',\
             'south of SR4','south of 60S']
subareas = ['south of SR4']
# ADD "south_of_SR4" and "south_of_60S"
# additionally, load masks for "south of SR4" and "south of 60
path_mask_SR4 = '/pscratch/sd/c/cnissen/HLRN_runs_postprocessed/masks/mask_south_of_transect_SR4.nc'
path_mask_60S = '/pscratch/sd/c/cnissen/HLRN_runs_postprocessed/masks/mask_south_of_transect_60S.nc'
path_mask_shelfBreak = '/pscratch/sd/c/cnissen/HLRN_runs_postprocessed/masks/mask_south_of_transect_shelf_break.nc'
fileSR4 = Dataset(path_mask_SR4)
file60S = Dataset(path_mask_60S)
file_shelfBreak = Dataset(path_mask_shelfBreak)
maskSR4 = fileSR4.variables['mask'][:]
mask60S = file60S.variables['mask'][:]
mask_shelfBreak = file_shelfBreak.variables['mask'][:]

print (np.min(shelf_ind),np.max(shelf_ind))


#-----
# plot map of where subareas are
#-----

from matplotlib.colors import ListedColormap

# make the color map:
cmap1 = ListedColormap(['white','darkblue','cornflowerblue','firebrick',\
                        'gold','orange','mediumpurple','violet','mediumturquoise'])
cmap1 = ListedColormap(['white',(51./255.,34./255.,136./255.),(136./255.,204./255.,238./255.),\
                  (68./255.,170./255.,153./255.),(17./255.,119./255.,51./255.),\
                  (153./255.,153./255.,51./255.),(221./255.,204./255.,119./255.),\
                   (204./255.,102./255.,119./255.),(136./255.,34./255.,85./255.)])

levels1 = np.arange(0,10,1)

plot_regions = False
if plot_regions:
    
    latlim = -60
    depth_threshold = 2000
    dpicnt = 200
    elem_no_nan = mesh.elem[mesh.no_cyclic_elem,:]
    
    fig8= plt.figure(num=18, figsize=(4,3), dpi=200, facecolor='w', edgecolor='k')
    map = Basemap(projection='spstere',boundinglat=-60,lon_0=-180,resolution='l')
    #map.drawcoastlines()
    map.drawcountries()#
    map.drawmapboundary(fill_color='0.7')
    x, y = map(mesh.x2, mesh.y2)
    shelf_ind[shelf_ind==0]=-999
    mm=plt.tricontourf(x, y, elem_no_nan[::], shelf_ind, levels = levels1, \
                        cmap=cmap1, extend='both') #,vmin=-4, vmax=4)
    cbar = plt.colorbar(orientation='horizontal',fraction=0.075, pad=0.03,shrink=0.75)# ,ticks=cticks)
    #cbar.cmap.set_under('white')
    cbar.remove() 
    #plt.draw()
    lw = 1.2

    topo_plot = np.copy(mesh.topo)
    topo_plot[cavities==1]=-999
    m2=plt.tricontour(x, y, elem_no_nan[::], topo_plot, levels = [2000],linewidths=[lw],colors=['black'])

    save_plot = False
    if save_plot:
        plt.savefig(savepath1+'Map_subareas_for_C_anth_analysis.png',\
                                            dpi = dpicnt, bbox_inches='tight') 
    plt.show()
    
    #----
    # MPAs
    #----
    all_mpas2 = np.zeros_like(all_mpas)
    all_mpas2[all_mpas>0]=1
    
    fig8= plt.figure(num=18, figsize=(4,3), dpi=200, facecolor='w', edgecolor='k')
    map = Basemap(projection='spstere',boundinglat=-60,lon_0=-180,resolution='l')
    #map.drawcoastlines()
    map.drawcountries()#
    map.drawmapboundary(fill_color='0.7')
    x, y = map(mesh.x2, mesh.y2)
    shelf_ind[shelf_ind==0]=-999
    mm=plt.tricontourf(x, y, elem_no_nan[::], all_mpas2, levels = levels1, \
                        cmap=cmap1, extend='both') #,vmin=-4, vmax=4)
    cbar = plt.colorbar(orientation='horizontal',fraction=0.075, pad=0.03,shrink=0.75)# ,ticks=cticks)
    #cbar.cmap.set_under('white')
    cbar.remove() 
    #plt.draw()
    lw = 1.2

    topo_plot = np.copy(mesh.topo)
    topo_plot[cavities==1]=-999
    m2=plt.tricontour(x, y, elem_no_nan[::], topo_plot, levels = [2000],linewidths=[lw],colors=['black'])

    save_plot = False
    if save_plot:
        plt.savefig(savepath1+'Map_subareas_MPAs.png',\
                                            dpi = dpicnt, bbox_inches='tight') 
    plt.show()
    #plt.close(fig8)

# Weddell
# eWeddell
# Prydz
# Adelie
# Ross
# Amundsen/Bellinghausen
# Antarctic Pensinsula



Subareas: south of 60°S...
0.0 7.0


In [4]:
#-----
# go through subregions and get how many grid cells are in each, without cavity
#-----

subregions = ['all_shelves','WeddellSea','eWeddellSea','Prydz','Adelie','RossSea','Amundsen_Bellingshausen','WAP',\
             'RossSea_MPA','RossSea_MPA_shelf','RossSea_MPA_openOcean',\
              'WeddellSea_MPA','WeddellSea_MPA_shelf','WeddellSea_MPA_openOcean',\
              'Orkney_MPA','AntarcticPeninsula_MPA',\
             'East_Antarctica_1_MPA','East_Antarctica_2_MPA','East_Antarctica_3_MPA','East_Antarctica_all_MPA',\
             'East_Antarctica_1_MPA_shelf','East_Antarctica_2_MPA_shelf',\
              'East_Antarctica_3_MPA_shelf','East_Antarctica_all_MPA_shelf',\
             'East_Antarctica_1_MPA_openOcean','East_Antarctica_2_MPA_openOcean',\
              'East_Antarctica_3_MPA_openOcean','East_Antarctica_all_MPA_openOcean',\
              'AntarcticPeninsula_MPA_shelf','AntarcticPeninsula_openOcean']

counter_nodes = True
num_nodes = np.zeros(len(subregions))
area_subregion = np.zeros(len(subregions))
if counter_nodes:
    for rr in range(0,len(subregions)):
        if subregions[rr] in ['all_shelves']:
            ind_regions = np.where(shelf_ind>0)[0]
        elif subregions[rr] in ['WeddellSea']:
            ind_regions = np.where(weddell==1)[0]
        elif subregions[rr] in ['eWeddellSea']:
            ind_regions = np.where(eWeddell==1)[0]
        elif subregions[rr] in ['Prydz']:
            ind_regions = np.where(prydz==1)[0]
        elif subregions[rr] in ['Adelie']:
            ind_regions = np.where(adelie==1)[0]
        elif subregions[rr] in ['RossSea']:
            ind_regions = np.where(ross==1)[0]
        elif subregions[rr] in ['Amundsen_Bellingshausen']:
            ind_regions = np.where(amundsen==1)[0]
        elif subregions[rr] in ['WAP']:
            ind_regions = np.where(wap==1)[0]
        elif subregions[rr] in ['RossSea_MPA']:
            ind_regions = np.where((mask_RossSea_MPA==1) & (cavities==0))[0]
        elif subregions[rr] in ['RossSea_MPA_shelf']:
            ind_regions = np.where((mask_RossSea_MPA==1) & (cavities==0) & (topo<=2000))[0]
        elif subregions[rr] in ['RossSea_MPA_openOcean']:
            ind_regions = np.where((mask_RossSea_MPA==1) & (cavities==0) & (topo>2000))[0]
        elif subregions[rr] in ['WeddellSea_MPA']:
            ind_regions = np.where((mask_WeddellSea_MPA==1) & (cavities==0))[0]
        elif subregions[rr] in ['WeddellSea_MPA_shelf']:
            ind_regions = np.where((mask_WeddellSea_MPA==1) & (cavities==0) & (topo<=2000))[0]
        elif subregions[rr] in ['WeddellSea_MPA_openOcean']:
            ind_regions = np.where((mask_WeddellSea_MPA==1) & (cavities==0) & (topo>2000))[0]
        elif subregions[rr] in ['AntarcticPeninsula_MPA']:
            ind_regions = np.where((mask_WAP_MPA==1) & (cavities==0))[0]
        elif subregions[rr] in ['AntarcticPeninsula_MPA_shelf']:
            ind_regions = np.where((mask_WAP_MPA==1) & (cavities==0) & (topo<=2000))[0]
        elif subregions[rr] in ['AntarcticPeninsula_openOcean']:
            ind_regions = np.where((mask_WAP_MPA==1) & (cavities==0) & (topo>2000))[0]
        elif subregions[rr] in ['Orkney_MPA']:
            ind_regions = np.where((mask_Orkney_MPA==1) & (cavities==0))[0]
        elif subregions[rr] in ['Orkney_MPA_shelf']:
            ind_regions = np.where((mask_Orkney_MPA==1) & (cavities==0) & (topo<=2000))[0]
        elif subregions[rr] in ['Orkney_MPA_openOcean']:
            ind_regions = np.where((mask_Orkney_MPA==1) & (cavities==0) & (topo>2000))[0]
        elif subregions[rr] in ['East_Antarctica_1_MPA']:
            ind_regions = np.where((mask_eastAA1_MPA==1) & (cavities==0))[0]
        elif subregions[rr] in ['East_Antarctica_1_MPA_shelf']:
            ind_regions = np.where((mask_eastAA1_MPA==1) & (cavities==0) & (topo<=2000))[0]
        elif subregions[rr] in ['East_Antarctica_1_MPA_openOcean']:
            ind_regions = np.where((mask_eastAA1_MPA==1) & (cavities==0) & (topo>2000))[0]
        elif subregions[rr] in ['East_Antarctica_2_MPA']:
            ind_regions = np.where((mask_eastAA2_MPA==1) & (cavities==0))[0]
        elif subregions[rr] in ['East_Antarctica_2_MPA_shelf']:
            ind_regions = np.where((mask_eastAA2_MPA==1) & (cavities==0) & (topo<=2000))[0]
        elif subregions[rr] in ['East_Antarctica_2_MPA_openOcean']:
            ind_regions = np.where((mask_eastAA2_MPA==1) & (cavities==0) & (topo>2000))[0]
        elif subregions[rr] in ['East_Antarctica_3_MPA']:
            ind_regions = np.where((mask_eastAA3_MPA==1) & (cavities==0))[0]
        elif subregions[rr] in ['East_Antarctica_3_MPA_shelf']:
            ind_regions = np.where((mask_eastAA3_MPA==1) & (cavities==0) & (topo<=2000))[0]
        elif subregions[rr] in ['East_Antarctica_3_MPA_openOcean']:
            ind_regions = np.where((mask_eastAA3_MPA==1) & (cavities==0) & (topo>2000))[0]
        elif subregions[rr] in ['East_Antarctica_all_MPA']:
            ind_regions = np.where(((mask_eastAA1_MPA==1) | (mask_eastAA2_MPA==1) | (mask_eastAA3_MPA==1) ) & (cavities==0))[0]
        elif subregions[rr] in ['East_Antarctica_all_MPA_shelf']:
            ind_regions = np.where(((mask_eastAA1_MPA==1) | (mask_eastAA2_MPA==1) | (mask_eastAA3_MPA==1) ) & (cavities==0) & (topo<=2000))[0]
        elif subregions[rr] in ['East_Antarctica_all_MPA_openOcean']:
            ind_regions = np.where(((mask_eastAA1_MPA==1) | (mask_eastAA2_MPA==1) | (mask_eastAA3_MPA==1) ) & (cavities==0) & (topo>2000))[0]
            
        num_nodes[rr] = int(ind_regions.shape[0])
        area_subregion[rr] = np.sum(area[ind_regions])
        
        
        

In [5]:

ind1 = subregions.index("RossSea_MPA")
ind2 = subregions.index("RossSea_MPA_shelf")
ind3 = subregions.index("RossSea_MPA_openOcean")
print(int(num_nodes[ind1]),int(num_nodes[ind2]),int(num_nodes[ind3]))

ind1 = subregions.index("AntarcticPeninsula_MPA")
ind2 = subregions.index("AntarcticPeninsula_MPA_shelf")
ind3 = subregions.index("AntarcticPeninsula_openOcean")
print(int(num_nodes[ind1]),int(num_nodes[ind2]),int(num_nodes[ind3]))

ind1 = subregions.index("RossSea_MPA")
ind2 = subregions.index("RossSea_MPA_shelf")
ind3 = subregions.index("RossSea_MPA_openOcean")
print(int(area_subregion[ind1]),int(area_subregion[ind2]),int(area_subregion[ind3]))

ind1 = subregions.index("AntarcticPeninsula_MPA")
ind2 = subregions.index("AntarcticPeninsula_MPA_shelf")
ind3 = subregions.index("AntarcticPeninsula_openOcean")
print(int(area_subregion[ind1]),int(area_subregion[ind2])/int(area_subregion[ind1]),int(area_subregion[ind3])/int(area_subregion[ind1]))


2464 2197 267
3632 3548 84
1592711708672 713687564288 879024144384
559666888704 0.732804330386088 0.26719575743758167


In [6]:
#-----
# go through subregions and get how many grid cells are in each, with cavity
#-----

subregions = ['all_shelves','WeddellSea','eWeddellSea','Prydz','Adelie','RossSea','Amundsen_Bellingshausen','WAP',\
             'RossSea_MPA','RossSea_MPA_shelf','RossSea_MPA_openOcean',\
              'WeddellSea_MPA','WeddellSea_MPA_shelf','WeddellSea_MPA_openOcean',\
              'Orkney_MPA','AntarcticPeninsula_MPA',\
             'East_Antarctica_1_MPA','East_Antarctica_2_MPA','East_Antarctica_3_MPA','East_Antarctica_all_MPA',\
             'East_Antarctica_1_MPA_shelf','East_Antarctica_2_MPA_shelf',\
              'East_Antarctica_3_MPA_shelf','East_Antarctica_all_MPA_shelf',\
             'East_Antarctica_1_MPA_openOcean','East_Antarctica_2_MPA_openOcean',\
              'East_Antarctica_3_MPA_openOcean','East_Antarctica_all_MPA_openOcean',\
              'AntarcticPeninsula_MPA_shelf','AntarcticPeninsula_openOcean']

counter_nodes = True
num_nodes_w_cavity = np.zeros(len(subregions))
if counter_nodes:
    for rr in range(0,len(subregions)):
        if subregions[rr] in ['all_shelves']:
            ind_regions = np.where(shelf_ind>0)[0]
        elif subregions[rr] in ['WeddellSea']:
            ind_regions = np.where(weddell==1)[0]
        elif subregions[rr] in ['eWeddellSea']:
            ind_regions = np.where(eWeddell==1)[0]
        elif subregions[rr] in ['Prydz']:
            ind_regions = np.where(prydz==1)[0]
        elif subregions[rr] in ['Adelie']:
            ind_regions = np.where(adelie==1)[0]
        elif subregions[rr] in ['RossSea']:
            ind_regions = np.where(ross==1)[0]
        elif subregions[rr] in ['Amundsen_Bellingshausen']:
            ind_regions = np.where(amundsen==1)[0]
        elif subregions[rr] in ['WAP']:
            ind_regions = np.where(wap==1)[0]
        elif subregions[rr] in ['RossSea_MPA']:
            ind_regions = np.where((mask_RossSea_MPA==1))[0]
        elif subregions[rr] in ['RossSea_MPA_shelf']:
            ind_regions = np.where((mask_RossSea_MPA==1) & (topo<=2000))[0]
        elif subregions[rr] in ['RossSea_MPA_openOcean']:
            ind_regions = np.where((mask_RossSea_MPA==1) & (cavities==0) & (topo>2000))[0]
        elif subregions[rr] in ['WeddellSea_MPA']:
            ind_regions = np.where((mask_WeddellSea_MPA==1))[0]
        elif subregions[rr] in ['WeddellSea_MPA_shelf']:
            ind_regions = np.where((mask_WeddellSea_MPA==1) & (topo<=2000))[0]
        elif subregions[rr] in ['WeddellSea_MPA_openOcean']:
            ind_regions = np.where((mask_WeddellSea_MPA==1) & (cavities==0) & (topo>2000))[0]
        elif subregions[rr] in ['AntarcticPeninsula_MPA']:
            ind_regions = np.where((mask_WAP_MPA==1))[0]
        elif subregions[rr] in ['AntarcticPeninsula_MPA_shelf']:
            ind_regions = np.where((mask_WAP_MPA==1) & (topo<=2000))[0]
        elif subregions[rr] in ['AntarcticPeninsula_openOcean']:
            ind_regions = np.where((mask_WAP_MPA==1) & (cavities==0) & (topo>2000))[0]
        elif subregions[rr] in ['Orkney_MPA']:
            ind_regions = np.where((mask_Orkney_MPA==1) & (cavities==0))[0]
        elif subregions[rr] in ['Orkney_MPA_shelf']:
            ind_regions = np.where((mask_Orkney_MPA==1) & (cavities==0) & (topo<=2000))[0]
        elif subregions[rr] in ['Orkney_MPA_openOcean']:
            ind_regions = np.where((mask_Orkney_MPA==1) & (cavities==0) & (topo>2000))[0]
        elif subregions[rr] in ['East_Antarctica_1_MPA']:
            ind_regions = np.where((mask_eastAA1_MPA==1) & (cavities==0))[0]
        elif subregions[rr] in ['East_Antarctica_1_MPA_shelf']:
            ind_regions = np.where((mask_eastAA1_MPA==1) & (cavities==0) & (topo<=2000))[0]
        elif subregions[rr] in ['East_Antarctica_1_MPA_openOcean']:
            ind_regions = np.where((mask_eastAA1_MPA==1) & (cavities==0) & (topo>2000))[0]
        elif subregions[rr] in ['East_Antarctica_2_MPA']:
            ind_regions = np.where((mask_eastAA2_MPA==1) & (cavities==0))[0]
        elif subregions[rr] in ['East_Antarctica_2_MPA_shelf']:
            ind_regions = np.where((mask_eastAA2_MPA==1) & (cavities==0) & (topo<=2000))[0]
        elif subregions[rr] in ['East_Antarctica_2_MPA_openOcean']:
            ind_regions = np.where((mask_eastAA2_MPA==1) & (cavities==0) & (topo>2000))[0]
        elif subregions[rr] in ['East_Antarctica_3_MPA']:
            ind_regions = np.where((mask_eastAA3_MPA==1) & (cavities==0))[0]
        elif subregions[rr] in ['East_Antarctica_3_MPA_shelf']:
            ind_regions = np.where((mask_eastAA3_MPA==1) & (cavities==0) & (topo<=2000))[0]
        elif subregions[rr] in ['East_Antarctica_3_MPA_openOcean']:
            ind_regions = np.where((mask_eastAA3_MPA==1) & (cavities==0) & (topo>2000))[0]
        elif subregions[rr] in ['East_Antarctica_all_MPA']:
            ind_regions = np.where(((mask_eastAA1_MPA==1) | (mask_eastAA2_MPA==1) | (mask_eastAA3_MPA==1) ))[0]
        elif subregions[rr] in ['East_Antarctica_all_MPA_shelf']:
            ind_regions = np.where(((mask_eastAA1_MPA==1) | (mask_eastAA2_MPA==1) | (mask_eastAA3_MPA==1) ) & (topo<=2000))[0]
        elif subregions[rr] in ['East_Antarctica_all_MPA_openOcean']:
            ind_regions = np.where(((mask_eastAA1_MPA==1) | (mask_eastAA2_MPA==1) | (mask_eastAA3_MPA==1) ) & (cavities==0) & (topo>2000))[0]
            
        num_nodes_w_cavity[rr] = int(ind_regions.shape[0])
            

In [9]:
#----
# How deep is it on average in Orkney MPA?
#----

ind_regions = np.where((mask_Orkney_MPA==1) & (cavities==0))[0]

print (topo[ind_regions])

print ('Min/Max topo:',np.min(topo[ind_regions]),np.max(topo[ind_regions]))



[2880.6 2580.6 3405.6 3220.6 2725.6 3806.9 3405.6 2320.6 2005.6 3806.9
 2320.6 3045.6 3405.6 2880.6 4025.6 2320.6 2205.6 2725.6 3806.9 3045.6
 4256.9 3806.9]
Min/Max topo: 2005.6 4256.9


In [14]:
#---
# print some numbers, WITHOUT cavity, SHELF only
#---
print ('')
print ('without cavity')
print ('')

for rr in [9,12,23]:#range(0,len(subregions)):
    print (subregions[rr]+': '+str(int(num_nodes[rr]))+' ('+\
                str(np.round(100*100*num_nodes[rr]/float(len(lon)))/100)+'% of all in mesh)')
    
print ('')


without cavity

RossSea_MPA_shelf: 2197 (3.03% of all in mesh)
WeddellSea_MPA_shelf: 1486 (2.05% of all in mesh)
East_Antarctica_all_MPA_shelf: 942 (1.3% of all in mesh)



In [15]:
#---
# print some numbers, WITHOUT cavity
#---
print ('')
print ('without cavity')
print ('')

for rr in [8,11,14,15,19]:#range(0,len(subregions)):
    print (subregions[rr]+': '+str(int(num_nodes[rr]))+' ('+\
                str(np.round(100*100*num_nodes[rr]/float(len(lon)))/100)+'% of all in mesh)')
    
print ('')
    


without cavity

RossSea_MPA: 2464 (3.4% of all in mesh)
WeddellSea_MPA: 2043 (2.82% of all in mesh)
Orkney_MPA: 22 (0.03% of all in mesh)
AntarcticPeninsula_MPA: 3632 (5.02% of all in mesh)
East_Antarctica_all_MPA: 1250 (1.73% of all in mesh)



In [16]:
#---
# print some numbers, WITH cavity
#---
print ('')
print ('with cavity')
print ('')

for rr in [8,11,14,15,19]:#range(0,len(subregions)):
    print (subregions[rr]+': '+str(int(num_nodes_w_cavity[rr]))+' ('+\
                str(np.round(100*100*num_nodes_w_cavity[rr]/float(len(lon)))/100)+'% of all in mesh)')
    
print ('')



with cavity

RossSea_MPA: 18682 (25.8% of all in mesh)
WeddellSea_MPA: 7977 (11.02% of all in mesh)
Orkney_MPA: 22 (0.03% of all in mesh)
AntarcticPeninsula_MPA: 7546 (10.42% of all in mesh)
East_Antarctica_all_MPA: 1649 (2.28% of all in mesh)



In [17]:
#---
# print some numbers, WITH cavity, shelf only
#---
print ('')
print ('with cavity')
print ('')

for rr in [9,12,23]:#range(0,len(subregions)):
    print (subregions[rr]+': '+str(int(num_nodes_w_cavity[rr]))+' ('+\
                str(np.round(100*100*num_nodes_w_cavity[rr]/float(len(lon)))/100)+'% of all in mesh)')
    
print ('')



with cavity

RossSea_MPA_shelf: 18415 (25.43% of all in mesh)
WeddellSea_MPA_shelf: 7417 (10.24% of all in mesh)
East_Antarctica_all_MPA_shelf: 1341 (1.85% of all in mesh)

